# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [ ]:
city_data = "city_data.csv"

cities_df = pd.read_csv(city_data)

cities_df.head()


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [ ]:
# Configure gmaps with API key.
gmaps.configure(api_key=g_key)

# Store 'Latitude' and 'Longitude' into  locations. 
locations = cities_df[["Lat", "Lng"]].astype(float)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
cities_df = cities_df.dropna()
humidity = cities_df["Humidity"].astype(float)
# len(humidity)

# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig
#The map shows the second time you run it? 

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [ ]:
vacation_df = cities_df.loc[(cities_df["Max Temp"] < 90) & (cities_df["Max Temp"] > 80)]
vacation_df = vacation_df.loc[vacation_df["Wind Speed"] > 10]
vacation_df = vacation_df.loc[vacation_df["Cloudiness"] == 0]
vacation_df = vacation_df.dropna()
vacation_df

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
        }
hotel_df = vacation_df

In [ ]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    long = row["Lng"]

    params["location"] = f"{lat},{long}"
    
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response["results"]
    
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)

# Display figure
# Create hotel symbol layer
hotel_layer = gmaps.symbol_layer(locations,
   info_box_content=hotel_info)
# Add the layer to the map
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig
#The markers show the second time you run it?